- 최근 20일의 오전,오후 60분봉 조회. 오전, 오후라는 건 시간대마다 다르기 때문에 시간대를 마음대로 변경할 수 있어야 한다. 기본 설정은 UTC-11.

# 오전, 오후 전략

- 매수 조건 : 오전 & 전일 오후 수익률 0% 이상 & 전일 오전 거래량 < 전일 오후 거래량
- 매수 비중 : 타겟 변동성 \* 어제 오전 변동성
- 매도 조건 : 오후
- 매도 비중 : 100%

1. 전일 60분 봉 전체 데이터를 조회. (총 24개)
2. 전일 오후 수익률 구하기. (오후 종가 - 오후 시가)가 0보다 큰지 확인한다.
3. 전일 오전, 오후 거래량 구해서 오후 거래량이 높은지 확인.
4. 투자 비중 구하기. 전일 오전 변동성이 3%인 경우. 타겟 변동성이 1%라면 투자 비중은 33%다. (1% / 3%) 내 자산의 33%를 투자하고 해당 자산이 3% 움직이면 내 자산의 최대 변동성은 1%다.

# 변동성 돌파 전략

- 매수 조건 : 오전
- 매수 비중 : (타겟 변동성 \* 어제 오전 변동성) / 3,5,10,20일 오전 이동평균선 스코어
- 매도 조건 : 오후
- 매도 비중 : 100%
- 매수 : 돌파 가격을 계산하고 매수 주문을 해당 금액으로 건다.

1. 최근 20일의 오전 데이터 조회
   - 최근 20일의 60분봉 데이터를 조회해서 원하는 시간대의 오전 데이터만 추출해서 일 별로 시가, 종가, 고가, 저가, 거래량을 구한다.
2. 전일 오전 레인지 구하기
3. 최근 20일 오전 평균 노이즈 비율 구하기. 최근 20일 오전 데이터에서 노이즈 비율을 구하고 평균을 낸다.
4. 3,5,10,20일 오전 이동평균선 구하기. 최근 20일 오전 데이터에서 3,5,10,20일 이동평균선을 구하고 전일 오전 종가와 비교 후 점수를 계산

- 매 시간마다 스케줄러를 돌면서 원하는 타임존의 현재 시간을 조회
  - 기록해둔 날짜와 달라졌다면 데이터를 다시 조회
  - 오전이라면 매수
  - 오후라면 매도

# 정적 자산 배분

- 매수 조건 : 종목의 보유 비중 조회해서 할당된 비중보다 낮으면 매수
- 매수 비중 : 할당 비중 - 보유 비중
- 매도 조건 : 종목의 보유 비중 조회해서 할당된 비중보다 높으면 매도
- 매도 비중 : 할당 비중 - 보유 비중


- 전략 할당 금액
- 종목 목록
- 세부 전략 목록

- 전략은 하나의 계좌를 가지고 있다.
- 전략은 할당된 비중을 가지고 있다. 계좌에 입금된 돈 중에서 전략에 할당된 비중만큼 투자한다.
- 전략은 트래이딩할 종목 목록을 가지고 있다. 각 종목 별로 비중이 있다. 종목 별 비중의 합은 1을 넘지 않는다.
- 종목마다 세부 전략 목록을 가지고 있다. 각 세부 전략 별로 비중이 있다. 세부 전략 별 비중의 합은 1을 넘지 않는다.


# 매수 종목 목록 추출

- 최근 3일간 거래대금 상위 5개 종목 추출


# 전략에 필요한 데이터

- 계좌 (잔고, 보유 종목 정보, 매수, 매도), DB 저장 연관관계 맺기
- 매매 종목 목록 :
- 할당 비중
- 타임존
- 전략 이름
- 마지막 실행 완료 시간


In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List

from src.account.domain.account import Account


@dataclass
class Symbol:
    symbol: str
    allocation: float


class Strategy(ABC):
    def __init__(
        self,
        name: str,
        account: Account,
        allocation: float,  # 전략 할당 비중 (0~1)
        symbols: List[Symbol],
        # target_volatility: float = 0.01,
        # timezone: str = "Etc/GMT+11",
    ):  # UTC-11
        self.name = name
        self.account = account
        self.allocation = allocation
        self.symbols = symbols
        self.last_update = None
        # self.timezone = pytz.timezone(timezone)
        # self.target_volatility = target_volatility

    def trade(self):
        if not self.should_trade():
            return

        for symbol in self.symbols:
            self.buy(symbol)
            self.sell(symbol)

    @abstractmethod
    def should_trade(self) -> bool:
        """거래 조건 확인"""
        pass

    @abstractmethod
    def buy(self, symbol: Symbol):
        """매수"""
        pass

    @abstractmethod
    def sell(self, symbol: Symbol):
        """매도"""
        pass

    @abstractmethod
    def should_buy(self, symbol: Symbol) -> bool:
        """매수 조건 확인"""
        pass

    @abstractmethod
    def should_sell(self, symbol: Symbol) -> bool:
        """매도 조건 확인"""
        pass

    @abstractmethod
    def calculate_buy_weight(self, symbol: Symbol) -> float:
        """매수 비중 계산"""
        pass

    @abstractmethod
    def calculate_sell_weight(self, symbol: Symbol) -> float:
        """매도 비중 계산"""
        pass

- 전략 생성
  - 전략 타입과 additional_field를 가지고 각 전략이 나뉜다.

- 정적자산배분
  - 거래 종목
  - 실행 주기

- 변동성돌파
  - 거래 종목
  - 목표 변동성
  - 타임존
  - 매수 비중
  - 돌파 기준 가격

- 오전오후
  - 거래 종목
  - 목표 변동성
  - 타임존
  

- 공통
  - id
  - 이름
  - account_id
  - 할당 비중
  - 마지막 실행 시간
  - 활성화 여부
  - 전략 타입
    - 정적자산배분
    - 변동성돌파
    - 오전오후

- 전략 CRUD 가능